""" Image Extraction """

In [1]:
# pip install selenium

In [1]:
import pandas as pd

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


In [3]:
def scroll_down_slowly():
    driver.execute_script("window.scrollBy(0, 500)")
    time.sleep(1)

def extract_data_from_page():
    brand_names = driver.find_elements(By.CSS_SELECTOR, 'div.product-productMetaInfo > h3')
    product_descriptions = driver.find_elements(By.XPATH, "//*[@class='product-product']")
    discount_prices = driver.find_elements(By.XPATH, '//*[@class="product-discountedPrice"]')
    original_prices = driver.find_elements(By.XPATH, '//span[@class="product-strike"]')
    discounts = driver.find_elements(By.XPATH, '//span[@class="product-discountPercentage"]')
    img_data = driver.find_elements(By.CSS_SELECTOR, 'img.img-responsive')
    image_elements = driver.find_elements(By.XPATH, '//source')

    data_list = []

    for i in range(len(brand_names)):
        data = {
            'Brand_Name': brand_names[i].text.strip() if i < len(brand_names) else '',
            'Product_Description' :product_descriptions[i].text.strip() if i < len(product_descriptions) else '',
            'Product_Detail_Description':  img_data[i].get_attribute('title') if i < len(img_data) else '' ,
            'Discounted_Price': discount_prices[i].text.strip() if i < len(discount_prices) else '',
            'Original_Price': original_prices[i].text.strip() if i < len(original_prices) else '',
            'Discount_Percent': discounts[i].text.strip() if i < len(discounts) else '',
            'Links_of_the_Product_Images': img_data[i].get_attribute('src') if i < len(img_data) else '',
            'Links_of_the_Product_Zoomed_Images': image_elements[i].get_attribute('srcset') if i < len(image_elements) else ''
            
        }
        data_list.append(data)

    return data_list

columns = ['Brand_Name', 'Product_Description','Product_Detail_Description', 'Discounted_Price', 'Original_Price', 'Discount_Percent', 'Links_of_the_Product_Images', 'Links_of_the_Product_Zoomed_Images']
data_list = []

driver = webdriver.Chrome()

for page_number in range(1, 4):
    page_url = f"https://www.myntra.com/men-tshirts?p={page_number}"
    driver.get(page_url)
    driver.maximize_window()


    for _ in range(10):
        scroll_down_slowly()

    page_data = extract_data_from_page()

    data_list.extend(page_data)


df = pd.DataFrame(data_list)


print("Final DataFrame:")
df.head()


Final DataFrame:


,Brand_Name,Product_Description,Product_Detail_Description,Discounted_Price,Original_Price,Discount_Percent,Links_of_the_Product_Images,Links_of_the_Product_Zoomed_Images
0,Stormborn,Oversized Pure Cotton T-shirt,Stormborn Graphic Printed Oversized Drop Shoul...,Rs. 549,Rs. 999,(45% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
1,BULLMER,Printed Cotton T-shirt,BULLMER Graphic Printed Oversized Cotton T-shirt,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
2,Elibolz,Oversized Cotton T-shirt,Elibolz Typography Printed Oversized Cotton T-...,Rs. 499,Rs. 799,(Rs. 300 OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
3,BULLMER,Oversized Cotton T-shirt,BULLMER Graphic Printed Round Neck Oversized C...,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
4,Stormborn,Pure Cotton Oversized T-shirt,Stormborn Graphic Printed Drop Shoulder Sleeve...,Rs. 539,Rs. 999,(46% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Brand_Name                          150 non-null    object
 1   Product_Description                 150 non-null    object
 2   Product_Detail_Description          150 non-null    object
 3   Discounted_Price                    150 non-null    object
 4   Original_Price                      150 non-null    object
 5   Discount_Percent                    150 non-null    object
 6   Links_of_the_Product_Images         150 non-null    object
 7   Links_of_the_Product_Zoomed_Images  150 non-null    object
dtypes: object(8)
memory usage: 9.5+ KB


""" Visualization """

In [5]:
df.head(2)

,Brand_Name,Product_Description,Product_Detail_Description,Discounted_Price,Original_Price,Discount_Percent,Links_of_the_Product_Images,Links_of_the_Product_Zoomed_Images
0,Stormborn,Oversized Pure Cotton T-shirt,Stormborn Graphic Printed Oversized Drop Shoul...,Rs. 549,Rs. 999,(45% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
1,BULLMER,Printed Cotton T-shirt,BULLMER Graphic Printed Oversized Cotton T-shirt,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."


In [7]:
import plotly.express as px
discount_counts = df["Discount_Percent"].value_counts()
filtered_df = df[df["Discount_Percent"].isin(discount_counts.index)]
fig = px.bar(filtered_df, x='Discount_Percent',title='Discounted Percent Counts')
fig.show()

In [8]:
discount_counts = df["Brand_Name"].value_counts()
filtered_df = df[df["Brand_Name"].isin(discount_counts.index)]
fig = px.bar(filtered_df, x='Brand_Name', title='Brand Name Counts')
fig.show()

""" Data Organization "and Store""

In [9]:
df.head(25)

,Brand_Name,Product_Description,Product_Detail_Description,Discounted_Price,Original_Price,Discount_Percent,Links_of_the_Product_Images,Links_of_the_Product_Zoomed_Images
0,Stormborn,Oversized Pure Cotton T-shirt,Stormborn Graphic Printed Oversized Drop Shoul...,Rs. 549,Rs. 999,(45% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
1,BULLMER,Printed Cotton T-shirt,BULLMER Graphic Printed Oversized Cotton T-shirt,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
2,Elibolz,Oversized Cotton T-shirt,Elibolz Typography Printed Oversized Cotton T-...,Rs. 499,Rs. 799,(Rs. 300 OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
3,BULLMER,Oversized Cotton T-shirt,BULLMER Graphic Printed Round Neck Oversized C...,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
4,Stormborn,Pure Cotton Oversized T-shirt,Stormborn Graphic Printed Drop Shoulder Sleeve...,Rs. 539,Rs. 999,(46% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
5,Roadster,Colourblocked T-shirt,Roadster Men Grey Melange & Black Colourblocke...,Rs. 279,Rs. 699,(60% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
6,HERE&NOW,Pure Cotton T-shirt,HERE&NOW Round Neck Pure Cotton T-shirt,Rs. 174,Rs. 699,(75% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
7,H&M,Cotton Pure Cotton T-shirt Regular Fit,HM Men White Solid Cotton Pure Cotton T-shirt ...,Rs. 299,Rs. 999,(70% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
8,Roadster,Colourblocked Round Neck Pure Cotton T-shirt,Roadster Men White Mustard Yellow Colourblock...,Rs. 899,Rs. 999,(10% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."
9,ADIDAS,Indian Cricket ODI Fan Jersey,ADIDAS Men Indian Cricket ODI Fan Polo Collar ...,Rs. 446,Rs. 1399,(Rs. 953 OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr..."


In [10]:
df.to_csv("myntra.csv")

In [11]:
import lancedb
from lancedb.embeddings import EmbeddingFunctionRegistry
import pandas as pd
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

In [12]:
registry = EmbeddingFunctionRegistry.get_instance()
clip = registry.get("open-clip").create()

In [13]:
clip

OpenClipEmbeddings(max_retries=7, name='ViT-B-32', pretrained='laion2b_s34b_b79k', device='cpu', batch_size=64, normalize=True)

In [14]:
registry = EmbeddingFunctionRegistry.get_instance()
clip = registry.get("open-clip").create(query_task_type="retrieval_query")
class MyntraImgModel(LanceModel):
    Brand_Name: str 
    Product_Description: str 
    Product_Detail_Description : str = clip.SourceField() 
    Discounted_Price: str
    Original_Price:str
    Discount_Percent : str
    Links_of_the_Product_Images: str = clip.SourceField()
    Links_of_the_Product_Zoomed_Images : str
    vector: Vector(clip.ndims()) = clip.VectorField()
db = lancedb.connect("~/.lancedb")
tbl = db.create_table("KurImages", schema=MyntraImgModel, mode="overwrite")

In [15]:
from PIL import Image


In [16]:
tbl.add(df)

100%|██████████| 22/22 [00:02<00:00,  7.83it/s]


In [20]:
tbl.to_pandas()

,Brand_Name,Product_Description,Product_Detail_Description,Discounted_Price,Original_Price,Discount_Percent,Links_of_the_Product_Images,Links_of_the_Product_Zoomed_Images,vector
0,Stormborn,Oversized Pure Cotton T-shirt,Stormborn Graphic Printed Oversized Drop Shoul...,Rs. 549,Rs. 999,(45% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.03619275, 0.10491923, -0.0071644518, -0.00..."
1,BULLMER,Printed Cotton T-shirt,BULLMER Graphic Printed Oversized Cotton T-shirt,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.024272757, 0.07638424, -0.07703896, -0.022..."
2,Elibolz,Oversized Cotton T-shirt,Elibolz Typography Printed Oversized Cotton T-...,Rs. 499,Rs. 799,(Rs. 300 OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.0042303726, 0.0066442364, -0.012772154, -0...."
3,BULLMER,Oversized Cotton T-shirt,BULLMER Graphic Printed Round Neck Oversized C...,Rs. 389,Rs. 1499,(74% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.0047468636, -0.047513563, -0.035395317, -0...."
4,Stormborn,Pure Cotton Oversized T-shirt,Stormborn Graphic Printed Drop Shoulder Sleeve...,Rs. 539,Rs. 999,(46% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.03878292, 0.11953718, -0.0632379, -0.00313..."
...,...,...,...,...,...,...,...,...,...
145,Blackberrys,Solid Polo Collar T-shirt,Blackberrys Solid Polo Collar T-shirt,Rs. 599,Rs. 1499,(60% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.018476851, 0.0048511187, 0.012291758, -0.05..."
146,Roadster,Printed Round Neck Pure Cotton T-shirt,Roadster Men Maroon White Printed Cotton Roun...,Rs. 759,Rs. 1999,(62% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.006931192, -0.087363765, -0.035250187, -0.0..."
147,HRX by Hrithik Roshan,Men High Neck T-shirt,HRX by Hrithik Roshan Men Solid High Neck T-shirt,Rs. 529,Rs. 999,(47% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.038396847, -0.116382435, -0.09668133, -0.00..."
148,STELLERS,Polo Collar Dry Fit T-shirt,STELLERS Polo Collar Dry Fit T-shirt,,,,"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.029387137, -0.012615847, -0.055797283, -0...."


In [21]:
rs_img = tbl.search("Roadster Men Colourblocked Polo Collar T-shirt").metric("cosine").limit(5).to_pandas()
rs_img

,Brand_Name,Product_Description,Product_Detail_Description,Discounted_Price,Original_Price,Discount_Percent,Links_of_the_Product_Images,Links_of_the_Product_Zoomed_Images,vector,_distance
0,STELLERS,Polo Collar Dry Fit T-shirt,STELLERS Polo Collar Dry Fit T-shirt,,,,"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.029387137, -0.012615847, -0.055797283, -0....",0.678659
1,Roadster,Striped Polo T-shirt,Roadster Men Charcoal Grey & Maroon Striped Po...,Rs. 329,Rs. 1099,(70% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.0045624, -0.10633086, -0.038434874, -0.048...",0.679199
2,Roadster,Printed Round Neck T-shirt,Roadster Men Black & Grey Printed Round Neck T...,Rs. 549,Rs. 999,(45% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[-0.03371084, -0.028055638, -0.036422458, -0.0...",0.683465
3,WROGN,Solid Polo Collar T-shirt,WROGN Men Burgundy Solid Polo Collar T-shirt,Rs. 404,Rs. 899,(55% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.00284867, -0.043762922, -0.072889425, -0.08...",0.688797
4,Mast & Harbour,Striped Polo Collar T-shirt,Mast & Harbour Men Teal Blue & Yellow Striped ...,Rs. 629,Rs. 1399,(55% OFF),"https://assets.myntassets.com/dpr_2,q_60,w_210...","\n https://assets.myntassets.com/f_webp,dpr...","[0.0025743772, -0.058943503, -0.1514616, -0.07...",0.694833


In [22]:
def find_best_passage(query):
    rs_img = tbl.search(query).metric("cosine").limit(5).to_pandas()
    img_path =  rs_img['Links_of_the_Product_Images'].tolist()
    return (img_path)

In [23]:
img = find_best_passage("Roadster Men Colourblocked Polo Collar T-shirt")

In [24]:
from IPython.display import display, Image
for image_url in img:
    display(Image(url=image_url))
